In [1]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from selenium.webdriver.firefox.options import Options
import pandas as pd
from shutil import move

In [35]:
options = Options()
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", "/home/kk/coursera_scraper/data")
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")
driver = webdriver.Firefox(options=options)

In [3]:
login_page_url = "https://www.coursera.org/learn/entrepreneurship-strategy?authMode=login"

In [36]:
driver.get(login_page_url)
time.sleep(2)


In [37]:
email_field = driver.find_element("id","email")
password_field = driver.find_element("id","password")
login_button = driver.find_element("xpath","//button[@class='_6dgzsvq css-1af0gyj']")
email_field.send_keys("<email>")
time.sleep(2)
password_field.send_keys("<password>")
time.sleep(2)
login_button.click()

In [73]:
url = "https://www.coursera.org/search?query=free&page=2&index=prod_all_launched_products_term_optimization&skills=Entrepreneurship&allLanguages=English"

In [ ]:
driver.maximize_window()
home_page = url
root_dir = "<path-to-save-scraped-data>"

for i in range(number_of_pages):
    
    # course_links = [course.get_attribute("href") for course in driver.find_elements("xpath","//a[@class='cds-119 cds-113 cds-115 cds-CommonCard-titleLink css-1cymd04 cds-142']")]
    links = driver.find_elements("xpath","//a")
    course_links = []
    for link in links:

        link = link.get_attribute("href")
        # print(link)
        if link == None:
            continue
        link_segment = link.split("https://www.coursera.org/")[-1]
        if link_segment.split("/")[0] == "learn":
            course_links.append(link)
            # print(link)
    if i == 0:
        course_links = course_links[6:]
    print("On page {}, Found {} courses..........".format(i+1,len(course_links)))
    for course_num,course_link in enumerate(course_links):
        course_name = course_link.split("/")[-1]
        print("Scraping for course: {},it's #{}.........".format(course_name,course_num))
        print("Course link: ",course_link)
        driver.get(course_link)
        time.sleep(10)
        destination_dir = os.path.join(root_dir,course_name)
        if not os.path.exists(destination_dir):
            os.makedirs(destination_dir)
        
        enrol_buttons = driver.find_elements("xpath","//span")
        for button in enrol_buttons:
            if button.text == "Enroll for Free":
                button.click()
                time.sleep(5)
                radio_buttons = driver.find_elements("xpath","//i[@class='cif-circle cif-stack-1x']")
                radio_buttons[-1].click()
                continue_button  = driver.find_element("xpath","//button[@class='primary continue-button cozy']")
                continue_button.click()
                time.sleep(10)
                driver.refresh()
        print("Going to the course..............")   
        time.sleep(10)
        go_tocourse = driver.find_elements("xpath","//span")
        for element in go_tocourse:
            if "Go To Course" in element.text:
                element.click()
                time.sleep(10)
                break
                

        weeks = [element.get_attribute("href") for element in driver.find_elements("xpath","//a[@data-track-component='nav_week']")]
        for week_num,week in enumerate(weeks):
            driver.get(week)
            time.sleep(10)
            video_navigation_links = [element.get_attribute("href") for element in driver.find_elements("xpath","//a[@data-track-component='item_link']")]
            print("Found {} video links.........".format(len(video_navigation_links)))
            for video_num,video_link in enumerate(video_navigation_links):
                print("Video num {} .....".format(video_num))
                driver.get(video_link)
                time.sleep(10)
                try:
                    download_dropdown = driver.find_element("id","downloads-dropdown-btn")
                    download_dropdown.click()
                    print("clicked on the dropdown...........")
                    time.sleep(3)
                    download_options = driver.find_elements("xpath","//span")
                    for download_option in download_options:
                        # print("we are here")
                        if "Transcript (English)" in download_option.text:
                            # print("here")
                            download_option.click()
                            print("Downloaded the video....")
                            break
                    time.sleep(2)
                    subtitle_file_name = course_name+"_week_"+str(week_num)+"_video_"+str(video_num)+".txt"
                    move(root_dir+"subtitle.txt",os.path.join(destination_dir,subtitle_file_name))
                except Exception as e:
                    print("Failed ..........")
                    continue
    driver.get(home_page)
    time.sleep(4)
    next_page = driver.find_element("xpath","//button[@aria-label='Next Page']")
    next_page.click()
    time.sleep(3)
    home_page = driver.current_url
                                              